# Download and import necessary libraries


In [8]:
# download necessary libraries - only do this once 
# import sys
# !{sys.executable} -m pip install psycopg2 
# !{sys.executable} -m pip install pandas 
# !{sys.executable} -m pip install numpy

# Import necessary libraries
import os 
import glob # used to search for a specific file pattern 
import psycopg2 # for connecting to postgres database
import numpy as np # great substitute for array 
import pandas as pd # to be able to use dataframe which is utilized during transformation(staging) stage 
from table_queries import * # to call the sql queries we made
from dotenv import load_dotenv, find_dotenv # to access the secret keys we've hidden in a separate file 

load_dotenv(find_dotenv()) # grab values inside env file

True

# Connect to database
#### create a function that grabs the values inside your data


In [9]:
# connecting to database 
conn = psycopg2.connect(host="localhost", port="5432",database="sparkifydb",user="postgres",password=os.getenv("DB_PASSWORD"))
cur = conn.cursor() # used to execute sql queries

# create a function that grabs the values inside your data 
def get_files(filepath):
    all_files = [] 
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files


# Process Song Data
#### processing song_data dataset to be able to insert values at songs and artists dimensional tables 

In [10]:
# extraction 
song_data = get_files("data/song_data") 

filepath = song_data[0]

# loading - creating a dataframe(think of this as temporary storage) to transform and clean the data 
df = pd.read_json(filepath, lines=True)
df.head()

# transformation - we are supposed to do some transformations but the dataset is already good enough, so yeah thats it 


,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


### 1. songs table 
#### performing etl to be able to insert values at songs table 

In [11]:
# extraction & transformation - grabbing data and only getting the columns we want
songs_df = df[["song_id","title","artist_id","year","duration"]]

for i,row in songs_df.iterrows():
    # loading - putting data inside the sparkifydb inside songs table 
    cur.execute(songs_dimension_insert, list(row))
    conn.commit() # we pretty much need to do this at every query execution :(


### 2. artists table
#### performing etl to be able to insert values at artists table 

In [12]:
# extraction & transformation - grabbing data and only getting the columns we want
artists_df = df[["artist_id","artist_name","artist_location","artist_latitude","artist_longitude"]]
artists_df.head()

for i,row in artists_df.iterrows():
    # loading - putting data inside the sparkifydb inside songs table 
    cur.execute(artists_dimension_insert, list(row))
    conn.commit() # we pretty much need to do this at every query execution :(

              song_id             title           artist_id  year   duration
0  SOMZWCG12A8C13C480  I Didn't Mean To  ARD7TVE1187B99BFB1     0  218.93179


# Process Log Data
#### processing log_data dataset to be able to insert values at users and time dimension tables

In [13]:
# extraction 
log_data = get_files("data/log_data") 

filepath = log_data[0]

# loading - creating a dataframe(think of this as temporary storage) to transform and clean the data 
df = pd.read_json(filepath, lines=True)
df.head(2)

# transformation - we are supposed to do some transformations but the dataset is already good enough, so yeah thats it 


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


### 3. time table
#### performing etl to be able to insert values at time table

In [23]:
# extraction 
df = df.query("page == 'NextSong'") # SELECT * FROM time WHERE page = 'NextSong'
df.head()

# transformation
# grab only the ts column, we'll be extracting the correct columns for time table from that column alone 
# to_datetime converts a timestamp into a human understandable datetime format
t = pd.to_datetime(df['ts'], unit='ms') 
df['ts'] = pd.to_datetime(df['ts'], unit='ms')

# dividing the datetime into different columns including hour,day,etc. 
time_data = list((t, t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.weekday)) # .dt is used to get only the specific part in a datetime | e.g. dt.week only grabs the week value from the datetime 
column_labels = list(('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')) # this is just another way of creating list

# convert the timestamp into a proper dataframe with columns that we want
# zip - returns a zip object where first item from the first argument is paired into the first item from the second argument, and so on.. item with no pair is ignored 
# dict - converts into a dictionary | if you pass a [(1, 'Geeks'), (2, 'For)] it would turn into {1: 'Geeks', 2: 'For'}
# from_dict - convert dictionary into rows | returns a dataframe | e.g. 
    # data = {'row_1': [3, 2, 1, 0], 'row_2': ['a', 'b', 'c', 'd']}
    # >>> pd.DataFrame.from_dict(data, orient='index')
    #        0  1  2  3
    # row_1  3  2  1  0
    # row_2  a  b  c  d
time_df =  pd.DataFrame.from_dict(dict(zip(column_labels, time_data))) # 
time_df.head()

# loading 
for i, row in time_df.iterrows():
    cur.execute(time_dimension_insert, list(row))
    conn.commit()

C:\Users\Admin\AppData\Local\Temp/ipykernel_13404/3282001989.py:12: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  time_data = list((t, t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.weekday)) # .dt is used to get only the specific part in a datetime | e.g. dt.week only grabs the week value from the datetime


### 4. users table
#### performing etl to be able to insert values at users table

In [25]:
# extraction & transformation - grabbing data and only getting the columns we want
users_df = df[["userId", "firstName", "lastName", "gender", "level"]]
users_df.head()

for i,row in users_df.iterrows():
    # loading 
    cur.execute(users_dimension_insert, list(row))
    conn.commit()